In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import trimesh
from numpy.random import default_rng

from magnet_pinn.generator.io import MeshWriter
from magnet_pinn.generator.phantoms import Tissue
from magnet_pinn.generator.samplers import PropertySampler
from magnet_pinn.generator.transforms import ToMesh, MeshesCutout, MeshesCleaning, Compose

First generate 3d objects we suppose to use

In [3]:
tissue = Tissue(
    num_children_blobs=3,
    initial_blob_radius=100,
    initial_blob_center_extent={
        "x": [-5, 5],
        "y": [-5, 5],
        "z": [-50, 50],
    },
    blob_radius_decrease_per_level=0.3,
    num_tubes=10,
    relative_tube_max_radius=0.1,
    relative_tube_min_radius=0.01
)
# Generate StructurePhantom with 3D geometric objects
raw_3d_structures = tissue.generate(seed=100112)
print(f"Generated phantom with:")
print(f"- Parent blob at {raw_3d_structures.parent.position}")
print(f"- {len(raw_3d_structures.children)} children blobs")
print(f"- {len(raw_3d_structures.tubes)} tubes")
raw_3d_structures

Generated phantom with:
- Parent blob at [ -0.63960743   2.6418829  -44.28188684]
- 3 children blobs
- 10 tubes


StructurePhantom(parent=Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.15249196584491434), empirical_min_offset=np.float64(-0.1538600159980968)), children=[Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.15851715989125542), empirical_min_offset=np.float64(-0.12862108003882172)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.15797518494698573), empirical_min_offset=np.float64(-0.15553681523389642)), Blob(relative_disruption_strength=0.1, empirical_max_offset=np.float64(0.1312429067042927), empirical_min_offset=np.float64(-0.14891046386944357))], tubes=[Tube(direction=array([-0.30717901,  0.77969378,  0.54563602]), height=10000), Tube(direction=array([-0.10379325, -0.73669741, -0.66820947]), height=10000), Tube(direction=array([ 0.63482952,  0.31298043, -0.7064239 ]), height=10000), Tube(direction=array([ 0.83007771, -0.26410249,  0.49114241]), height=10000), Tube(direction=array([-0.18380314,  0.53295333, -0.8

Process 3d obejcts into the meshes

In [4]:
workflow = Compose([
    ToMesh(),
    MeshesCutout(),
    MeshesCleaning()
])
meshes = workflow(raw_3d_structures)
print(f"Processed phantom with:")
print(f"- Parent mesh with {len(meshes.parent.vertices)} vertices")
print(f"- {len(meshes.children)} children meshes")
print(f"- {len(meshes.tubes)} tube meshes")
meshes

Processed phantom with:
- Parent mesh with 41807 vertices
- 3 children meshes
- 10 tube meshes


MeshPhantom(parent=<trimesh.Trimesh(vertices.shape=(41807, 3), faces.shape=(83666, 3))>, children=[<trimesh.Trimesh(vertices.shape=(9558, 3), faces.shape=(19108, 3))>, <trimesh.Trimesh(vertices.shape=(9856, 3), faces.shape=(19712, 3))>, <trimesh.Trimesh(vertices.shape=(10267, 3), faces.shape=(20530, 3))>], tubes=[<trimesh.Trimesh(vertices.shape=(229, 3), faces.shape=(454, 3))>, <trimesh.Trimesh(vertices.shape=(288, 3), faces.shape=(572, 3))>, <trimesh.Trimesh(vertices.shape=(230, 3), faces.shape=(456, 3))>, <trimesh.Trimesh(vertices.shape=(214, 3), faces.shape=(424, 3))>, <trimesh.Trimesh(vertices.shape=(127, 3), faces.shape=(250, 3))>, <trimesh.Trimesh(vertices.shape=(154, 3), faces.shape=(304, 3))>, <trimesh.Trimesh(vertices.shape=(156, 3), faces.shape=(308, 3))>, <trimesh.Trimesh(vertices.shape=(164, 3), faces.shape=(324, 3))>, <trimesh.Trimesh(vertices.shape=(120, 3), faces.shape=(236, 3))>, <trimesh.Trimesh(vertices.shape=(192, 3), faces.shape=(380, 3))>])

In [5]:
trimesh.boolean.union(
    meshes.tubes + meshes.children,
    engine='manifold',
).show()

Sample properties for each mesh

In [8]:
prop_sampler = PropertySampler(
    {
        "density": {
            "min": 400,
            "max": 2000
        },
        "conductivity": {
            "min": 0,
            "max": 2.5
        },
        "permittivity": {
            "min": 1,
            "max": 71
        }
    }
)
prop = prop_sampler.sample_like(meshes, rng=default_rng())
prop

PropertyPhantom(parent=PropertyItem(conductivity=0.9372376958692084, permittivity=21.147927593710452, density=1462.0673418521603), children=[PropertyItem(conductivity=2.297586983345192, permittivity=26.267790558908978, density=1905.9439432920478), PropertyItem(conductivity=2.0426668688943823, permittivity=36.08675490904491, density=1328.4644615225398), PropertyItem(conductivity=1.6938149062760421, permittivity=70.6132795331878, density=1628.1156558734615)], tubes=[PropertyItem(conductivity=2.3986858727618303, permittivity=14.481033714773433, density=1760.3853107912555), PropertyItem(conductivity=1.992517136688893, permittivity=36.70431598975365, density=1334.2366125601004), PropertyItem(conductivity=0.3111927702183376, permittivity=62.43431691012753, density=1326.8835921841242), PropertyItem(conductivity=2.0249840149030853, permittivity=70.60210827945863, density=901.4034138504105), PropertyItem(conductivity=1.8341341495018304, permittivity=42.85788617849157, density=1480.040668383038)

Now export meshes as `.stl` files and also pandas dataframe with columns `file`, `permittivity`, `conductivity`, `density` into the `materials.txt` file in the `.csv` format.

In [9]:
writer = MeshWriter("../gen_data/raw/tissue_meshes")
writer.write(meshes, prop)